In [1]:
from dotenv import load_dotenv
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime
from langgraph.store.memory import InMemoryStore
from langchain.chat_models import init_chat_model

load_dotenv()

True

BUILD A CONTEXT CLASS

In [2]:
@dataclass
class Context:
    """User-specific context passed at runtime."""
    user_id: str
    user_name: str

TOOLS

In [3]:
@tool
def add_todo(task: str, runtime: ToolRuntime[Context]) -> str:
    """ Add new todo task for the current user """

    user_id = runtime.context.user_id
    user_name = runtime.context.user_name

    todos =[]
    if runtime.store:
        stored_data = runtime.store.get(("todos",), user_id)
        if stored_data:
            todos = stored_data.value.get(("tasks"), [])
    
    new_todo = {
        "id" : len(todos) +1,
        "task" : task,
        "completed" : False
    }

    todos.append(new_todo)

    if runtime.store:
        runtime.store.put(("todos",), user_id, {"tasks": todos})
    
    return f"Added todo for {user_name}: '{task}' (ID: {new_todo['id']})"

@tool
def view_todos(runtime: ToolRuntime[Context]) -> str:
    """View all todos for the current user."""
    user_id = runtime.context.user_id
    user_name = runtime.context.user_name
    
    # Get todos from store
    if runtime.store:
        stored_data = runtime.store.get(("todos",), user_id)
        if stored_data:
            todos = stored_data.value.get("tasks", [])
            
            if not todos:
                return f"{user_name}, you have no todos yet!"
            
            result = f"{user_name}'s Todo List:\n"
            for todo in todos:
                status = "✓" if todo["completed"] else "○"
                result += f"{status} {todo['id']}. {todo['task']}\n"
            return result
    
    return f"{user_name}, you have no todos yet!"

@tool
def complete_todo(todo_id: int, runtime: ToolRuntime[Context]) -> str:
    """Mark a todo as completed.
    
    Args:
        todo_id: The ID of the todo to mark as complete
    """
    user_id = runtime.context.user_id
    user_name = runtime.context.user_name
    
    if runtime.store:
        stored_data = runtime.store.get(("todos",), user_id)
        if stored_data:
            todos = stored_data.value.get("tasks", [])
            
            # Find and update the todo
            for todo in todos:
                if todo["id"] == todo_id:
                    todo["completed"] = True
                    runtime.store.put(("todos",), user_id, {"tasks": todos})
                    return f"Marked '{todo['task']}' as completed for {user_name}!"
            
            return f"Todo with ID {todo_id} not found."
    
    return f"{user_name}, you have no todos yet!"

@tool
def delete_todo(todo_id: int, runtime: ToolRuntime[Context]) -> str:
    """Delete a todo task.
    
    Args:
        todo_id: The ID of the todo to delete
    """
    user_id = runtime.context.user_id
    user_name = runtime.context.user_name
    
    if runtime.store:
        stored_data = runtime.store.get(("todos",), user_id)
        if stored_data:
            todos = stored_data.value.get("tasks", [])
            
            # Find and remove the todo
            original_length = len(todos)
            todos = [t for t in todos if t["id"] != todo_id]
            
            if len(todos) < original_length:
                runtime.store.put(("todos",), user_id, {"tasks": todos})
                return f"Deleted todo {todo_id} for {user_name}!"
            
            return f"Todo with ID {todo_id} not found."
    
    return f"{user_name}, you have no todos yet!"

CREATE YOUR TODO AGENT

In [4]:
def create_todo_agent():
    """Create the todo assistant agent with Runtime support."""
    
    store = InMemoryStore()
    model = init_chat_model("gpt-4o")

    agent = create_agent(
        model=model,
        tools=[add_todo, view_todos, complete_todo, delete_todo],
        system_prompt="""You are a helpful todo assistant. 
        Help users manage their todo lists using the available tools.
        Always be friendly and confirm actions.""",
        context_schema=Context, 
        store=store
    )
    
    return agent

In [5]:
agent = create_todo_agent()
response = agent.invoke(
        {"messages": [{"role": "user", "content": "Add 'Win MelbourneGP' to my list"}]},
        context=Context(user_id="001", user_name="Kimi")
    )

print(f"Todo Assistant: {response['messages'][-1].content}\n")

Todo Assistant: I've added "Win MelbourneGP" to your todo list. If you need help with anything else, feel free to let me know!



In [6]:
store = agent.store
items = store.search(("todos",))
items

[Item(namespace=['todos'], key='001', value={'tasks': [{'id': 1, 'task': 'Win MelbourneGP', 'completed': False}]}, created_at='2026-01-06T04:13:30.500028+00:00', updated_at='2026-01-06T04:13:30.500030+00:00', score=None)]

In [7]:
response = agent.invoke(
        {"messages": [{"role": "user", "content": "Add 'call Max for Turn 5,7 tips at silverstone' to my list"}]},
        context=Context(user_id="001", user_name="Kimi")
    )
print(f"Todo Assistant: {response['messages'][-1].content}\n")

Todo Assistant: I've added the task "call Max for Turn 5,7 tips at silverstone" to your todo list. If you need anything else, feel free to let me know!



In [8]:
response = agent.invoke(
        {"messages": [{"role": "user", "content": "Show me my todo list how many are completed"}]},
        context=Context(user_id="001", user_name="Kimi")
    )
print(f"Todo Assistant: {response['messages'][-1].content}\n")

Todo Assistant: Here's your current todo list:

1. Win MelbourneGP
2. Call Max for Turn 5,7 tips at Silverstone

None of these tasks are marked as completed yet. Let me know if you need help managing them!



In [9]:
response = agent.invoke(
        {"messages": [{"role": "user", "content": "I have won melbourne gp so please mark that as done"}]},
        context=Context(user_id="001", user_name="Kimi")
    )
print(f"Todo Assistant: {response['messages'][-1].content}\n")

Todo Assistant: Congratulations, Kimi! I've marked "Win MelbourneGP" as completed on your todo list. 🏆 If there's anything else you need help with, just let me know!



In [10]:
response = agent.invoke(
        {"messages": [{"role": "user", "content": "Show me my todo list how many are completed"}]},
        context=Context(user_id="001", user_name="Kimi")
    )
print(f"Todo Assistant: {response['messages'][-1].content}\n")

Todo Assistant: Here's your current todo list:

1. **Win MelbourneGP** - Completed
2. **Call Max for Turn 5,7 tips at silverstone** - Not Completed

You have 1 task completed out of 2 tasks. Would you like to do anything else with your todo list?



In [11]:
response = agent.invoke(
        {"messages": [{"role": "user", "content": "please delete all tasks from my todo"}]},
        context=Context(user_id="001", user_name="Kimi")
    )
print(f"Todo Assistant: {response['messages'][-1].content}\n")

Todo Assistant: All tasks have been successfully deleted from your todo list. If you need to add new tasks or require further assistance, feel free to ask!



In [12]:
response = agent.invoke(
        {"messages": [{"role": "user", "content": "Show me my todo list how many are completed"}]},
        context=Context(user_id="001", user_name="Kimi")
    )
    
print(f"Todo Assistant: {response['messages'][-1].content}\n")

Todo Assistant: It looks like you don't have any todos yet. If you'd like to add tasks to your list, just let me know!

